# **Ayudantía 12 - Ejercicios C2**
## Etienne Rojas

Ante cualquier duda o posible corrección, por favor mandar un correo a `etienne.rojas@sansano.usm.cl` 


---

# **Contexto**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
import ipywidgets as widgets

In [ ]:
f = lambda x: np.sin(x) * np.cos(x)

def plot_function(n):
    x = np.linspace(0, 10, n)
    y = f(x)

    plt.figure(figsize=(8, 4))
    plt.plot(x, y, '.')
    # plt.plot(x, y, '.-')
    plt.title(f'n = {n}')
    plt.grid(True)
    plt.xlabel('x')
    plt.ylabel('f(x) = sin(x) * cos(x)')
    plt.show()

interact(plot_function, n=IntSlider(value=20, min=10, max=100, step=10, description='Puntos'))


interactive(children=(IntSlider(value=20, description='Puntos', min=10, step=10), Output()), _dom_classes=('wi…

<function __main__.plot_function(n)>

---

# **Código Estudiantes**

In [5]:
def newton_1d(f,fp,x0, tol=1e-10, max_iter=100):
    xi = x0 - f(x0)/fp(x0)

    for i in range(max_iter):
        x_next = xi - f(x0)/fp(x0)
        if x_next-xi < tol:
            return x_next
        xi = x_next
    return x_next

In [14]:
def Newton_1D(f, fp, x0, tol=1e-6, max_iter=100):
    """
    Newton's method for finding roots of a function in 1D.
    
    # Input:
    - f (callable)     : Function for which to find the root
    - fp (callable)    : Derivative of the function
    - x0 (float)       : Initial guess
    - tol (float)      : Tolerance for convergence
    - max_iter (int)   : Maximum number of iterations
    
    # Output:
    - x (float)        : Approximate root of the function
    """
    x = x0
    for i in range(max_iter):
        fx = f(x)
        if abs(fx) < tol:
            break
        fpx = fp(x)
        if abs(fpx) < 1e-15:
            break
        x = x - fx / fpx
    return x

def find_next_point_STD(xi, g, gp, l, b, q):
    """
    Find the next point in the sequence based on the given parameters.
    
    # Input:
    - xi (float)       : Point x_i
    - g (callable)     : Function g(x)
    - gp (callable)    : Derivative of g(x)
    - l (float)        : Arclength target
    - b (float)        : Right limit of the interval of the domain of g()
    - q (float)        : Number of poins for numerical integration
    
    # Output:
    - x_next (float)   : Next point in the sequence
    - g_next (float)   : Value of g at the next point
    """
    # PASO 1: COMPUTAR LOS PESOS Y NODOS GAUSSIANOS
    x_gauss, w_gauss = np.polynomial.legendre.leggauss(q)

    # PASO 2: DEFINIR h(x) = √(1 + [g'(x)]^2)
    h = lambda x: np.sqrt(1 + (np.power(gp(x), 2)))

    # PASO 3: Definir f(δ) = int {xi}^{xi+δ} h(x) dx - l
    def f(delta):
        # transformar nodos [xi + xi+δ]
        t = xi + delta * (x_gauss + 1) / 2
        h_values = h(t) 
        # calcular la integral usando la cuadratura Gaussiana 
        integral = np.dot(w_gauss, h_values) * (delta / 2)
        return integral - l
    
    # PASO 2: construir la funcion f_prima
    def fp(delta):
        return h(xi+delta)

    
    delta = Newton_1D(f,fp,l/2)

    if (xi + delta <= b):
        x_next = xi+delta
        g_next = g(x_next)
    else:
        x_next = b
        g_next = g(b)
    

    return x_next, g_next


---

# **Código Pauta**

In [ ]:
# def gaussian_nodes_and_weights(q):
    

In [8]:
def Newton_1D(f, fp, x0, tol=1e-6, max_iter=100):
    """
    Newton's method for finding roots of a function in 1D.
    
    # Input:
    - f (callable)     : Function for which to find the root
    - fp (callable)    : Derivative of the function
    - x0 (float)       : Initial guess
    - tol (float)      : Tolerance for convergence
    - max_iter (int)   : Maximum number of iterations
    
    # Output:
    - x (float)        : Approximate root of the function
    """
    x = x0
    for i in range(max_iter):
        fx = f(x)
        if abs(fx) < tol:
            break
        fpx = fp(x)
        if abs(fpx) < 1e-15:
            break
        x = x - fx / fpx
    return x

In [15]:
def find_next_point(xi, g, gp, l, b, q):
    """
    Find the next point in the sequence based on the given parameters.
    
    # Input:
    - xi (float)       : Point x_i
    - g (callable)     : Function g(x)
    - gp (callable)    : Derivative of g(x)
    - l (float)        : Arclength target
    - b (float)        : Right limit of the interval
    - q (float)        : Number of poins for numerical integration
    
    # Output:
    - x_next (float)   : Next point in the sequence
    - g_next (float)   : Value of g at the next point
    """
    # PASO 1: COMPUTAR LOS PESOS Y NODOS GAUSSIANOS
    x_gauss, w_gauss = np.polynomial.legendre.leggauss(q)

    # PASO 2: DEFINIR h(x) = √(1 + [g'(x)]^2)
    h = lambda x: np.sqrt(1 + (np.power(gp(x), 2)))

    # PASO 3: Definir f(δ) = int {xi}^{xi+δ} h(x) dx - l
    def f(delta):
        # transformar nodos [xi + xi+δ]
        t = xi + delta * (x_gauss + 1) / 2
        h_values = h(t) 
        # calcular la integral usando la cuadratura Gaussiana 
        integral = np.dot(w_gauss, h_values) * (delta / 2)
        return integral - l
    
    # PASO 4: Definir f'(δ) = h(xi + δ) (TFC)
    fp = lambda delta: h(xi + delta) 
    
    # PASO 5: Resolver f(δ) = 0 con Newton
    r = Newton_1D(f, fp, x0=l/2)
    
    # PASO 6: Verificar limites y calcular resultado
    if xi + r <= b: 
        x_next = xi + r
    else:
        x_next = b
    
    g_next = g(x_next)
    return x_next, g_next


In [ ]:
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown

def generate_arc_length_points(a, b, g, gp, l_arc, q=10):
    """Genera secuencia de puntos con arc-length constante"""
    points = [a]
    current = a
    while current < b:
        next_point, _ = find_next_point(current, g, gp, l_arc, b, q)
        
        if np.isclose(next_point, current, atol=1e-10) or next_point <= current:
            break
        points.append(next_point)
        current = next_point
    

    if not np.isclose(points[-1], b):
        points.append(b)
    
    return np.array(points)


def f1(x):
    return np.sin(x) * np.cos(x)

def fp1(x):
    return np.cos(2*x)

def f2(x):
    return np.exp(-x/2) * np.sin(3*x)

def fp2(x):
    return np.exp(-x/2) * (3*np.cos(3*x) - 0.5*np.sin(3*x))

def f3(x):
    return 0.1 * (x - 2)**3 - 0.5*(x - 5)**2 + 2

def fp3(x):
    return 0.3*(x - 2)**2 - (x - 5)


def plot_comparison(n_points=20, l_arc=0.5, function='sin(x)cos(x)'):
    if function == 'sin(x)cos(x)':
        f, fp, a, b = f1, fp1, 0, 10
    elif function == 'e^{-x/2}sin(3x)':
        f, fp, a, b = f2, fp2, 0, 10
    else:
        f, fp, a, b = f3, fp3, 0, 10
    
    # Generar puntos equiespaciados
    x_eq = np.linspace(a, b, n_points)
    y_eq = f(x_eq)
    
    # Generar puntos con arc-length constante
    x_arc = generate_arc_length_points(a, b, f, fp, l_arc)
    y_arc = f(x_arc)

    # Calcular valores reales de f() para comparación
    x_real = np.linspace(a, b, 1000)
    y_real = f(x_real)
    
    # Crear figura con cuadrícula 2x2
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    plt.subplots_adjust(hspace=0.3, wspace=0.3)
    
    # Subplot (0,0): Scatter de f() con puntos equidistantes
    axs[0, 0].plot(x_eq, y_eq, 'bo-', alpha=0.7, label='Equiespaciados')
    axs[0, 0].plot(x_real, y_real, alpha=0.5, label='f(x) real', color='gray')
    axs[0, 0].set_title(f'Puntos equiespaciados (n={n_points})')
    axs[0, 0].set_xlabel('x')
    axs[0, 0].set_ylabel('f(x)')
    axs[0, 0].grid(True)
    
    # Subplot (0,1): Scatter de f() con puntos arclen equidistantes
    axs[0, 1].plot(x_arc, y_arc, 'ro-', alpha=0.7, label='Arc-length constante')
    axs[0, 1].plot(x_real, y_real, alpha=0.5, label='f(x) real', color='gray')
    axs[0, 1].set_title(f'Arc-length constante (n={len(x_arc)}, l={l_arc:.2f})')
    axs[0, 1].set_xlabel('x')
    axs[0, 1].set_ylabel('f(x)')
    axs[0, 1].grid(True)
    
    # Subplot (1,0): Distribución de puntos en el eje x
    axs[1, 0].plot(x_eq, np.zeros_like(x_eq), 'bo', alpha=0.7, label='Equiespaciados')
    axs[1, 0].plot(x_arc, np.ones_like(x_arc), 'ro', alpha=0.7, label='Arc-length')
    axs[1, 0].set_title('Distribución de puntos en el eje x')
    axs[1, 0].set_yticks([0, 1])
    axs[1, 0].set_yticklabels(['Equiespaciados', 'Arc-length'])
    axs[1, 0].set_xlabel('x')
    axs[1, 0].set_ylim(-0.5, 1.5)
    axs[1, 0].grid(True)
    axs[1, 0].legend()
    
    # Subplot (1,1): Vacío
    axs[1, 1].axis('off') 

    fig.suptitle(f'Comparación de métodos para: {function}', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    plt.show()

# Interfaz interactiva
interact(plot_comparison,
         n_points=IntSlider(value=20, min=5, max=100, step=5, description='Puntos equiespaciados'),
         l_arc=FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description='Longitud arco'),
         function=Dropdown(options=['sin(x)cos(x)', 'e^{-x/2}sin(3x)', 'Polinomio cúbico'], 
                           value='sin(x)cos(x)', description='Función'))

interactive(children=(IntSlider(value=20, description='Puntos equiespaciados', min=5, step=5), FloatSlider(val…

<function __main__.plot_comparison(n_points=20, l_arc=0.5, function='sin(x)cos(x)')>